# Modeling for CBS League
I am going to perform some analysis on players to see if I can gleam some analysis to help my team.

In [238]:
%load_ext autoreload
%autoreload 2
# import
import pandas as pd
import numpy as np
import sqlite3

con = sqlite3.Connection("/Users/tstuessi/Dev/ff-analysis/fantasyfootball.db")

from ff_utils import MultiColumnLabelEncoder

# scikit learn
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error

from ff_ruleset import Ruleset

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [278]:
# training data
with open("CREATE_DATA_MART_FULL_YEAR.sql", 'r') as f:
    train_query = f.read()

data_df = pd.read_sql(train_query, con)

In [279]:
data_df.head()

,playerid,team,position,defense_ast,defense_ffum,defense_int,defense_sk,defense_tkl,fumbles_lost,fumbles_rcv,...,_rushing_lngtd,_rushing_tds,_rushing_twopta,_rushing_twoptm,_rushing_yds,target,ypc,pass_cmp_ratio,ypr,passer_rating
0,00-0004091,SF,K,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,14.0,3.0,0.0,0.0,808.0,97.0,0.0,0.0,0.000000,0.0
1,00-0016919,IND,K,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,257.0,155.0,0.0,0.0,0.000000,0.0
2,00-0019646,OAK,K,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,72.0,142.0,0.0,0.0,0.000000,0.0
3,00-0019714,HOU,P,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,12.0,4.0,1.0,1.0,939.0,0.0,0.0,0.0,0.000000,0.0
4,00-0020337,BAL,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,22.0,3.0,0.0,0.0,716.0,0.0,0.0,0.0,14.565217,0.0


In [280]:
data_df[data_df["target"] > 100].describe()

,defense_ast,defense_ffum,defense_int,defense_sk,defense_tkl,fumbles_lost,fumbles_rcv,fumbles_tot,fumbles_trcv,fumbles_yds,...,_rushing_lngtd,_rushing_tds,_rushing_twopta,_rushing_twoptm,_rushing_yds,target,ypc,pass_cmp_ratio,ypr,passer_rating
count,86.0,86.0,86.0,86.0,86.0,86.000000,86.000000,86.000000,86.000000,86.000000,...,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000
mean,0.0,0.0,0.0,0.0,0.0,1.093023,0.872093,2.453488,0.883721,-2.511628,...,35.081395,3.569767,0.197674,0.023256,547.348837,157.459302,2.681384,0.178540,7.178965,108.856318
std,0.0,0.0,0.0,0.0,0.0,1.427692,1.344238,3.212684,1.349672,6.852641,...,56.001726,3.549783,0.504562,0.151599,454.740178,47.306150,3.383324,0.302558,9.350107,188.707093
min,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,-34.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,101.500000,-1.000000,0.000000,-6.000000,-185.416667
25%,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,122.000000,124.625000,0.000000,0.000000,0.000000,0.000000
50%,0.0,0.0,0.0,0.0,0.0,0.500000,0.000000,1.000000,0.000000,0.000000,...,6.000000,2.000000,0.000000,0.000000,370.000000,142.500000,2.091667,0.000000,5.738359,0.000000
75%,0.0,0.0,0.0,0.0,0.0,2.000000,1.750000,4.000000,2.000000,0.000000,...,50.000000,7.000000,0.000000,0.000000,888.250000,181.625000,4.455788,0.575608,12.146635,355.153766
max,0.0,0.0,0.0,0.0,0.0,6.000000,5.000000,14.000000,5.000000,6.000000,...,232.000000,12.000000,2.000000,1.000000,1592.000000,301.000000,22.250000,1.000000,45.000000,539.583333


In [281]:
msk = (data_df["target"] != 0) & (data_df["position"].isin(Ruleset.offensive_players()))
X = data_df[msk].drop(["playerid", "name", "target"] + Ruleset.defensive_players(), axis=1, errors="ignore")
y = data_df[msk]["target"]

In [288]:
cat_indices = [X.columns.get_loc("position"), X.columns.get_loc("team")]
pip_list = [('lbl_encoder', MultiColumnLabelEncoder(["position", "team"])),
            ('ohe', OneHotEncoder(categorical_features=cat_indices, sparse=False)),
            ('var', VarianceThreshold()),
            #('pca', PCA()),
            #('scaler', MinMaxScaler()),
            ('scaler', StandardScaler()),
            ('model', GradientBoostingRegressor(n_estimators=300, max_depth=1))]
            #('model', LinearRegression())]
model = Pipeline(pip_list)

In [289]:
np.sqrt(np.abs(cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=10))).mean()

47.48988006987849

In [274]:
model.fit(X, y)
preds = model.predict(X)

preds_df = pd.DataFrame()
preds_df["name"] = data_df[msk]["name"]
preds_df["position"] = data_df[msk]["position"]
preds_df["true"] = y
preds_df["preds"] = preds

In [275]:
preds_df[preds_df["position"] == 'RB'].sort_values(by="preds", ascending=False)

,name,position,true,preds
644,D.Freeman,RB,34.0,24.850515
174,L.McCoy,RB,26.0,17.486455
364,T.Cadet,RB,20.0,13.343037
785,D.Johnson,RB,17.0,12.623798
114,J.Stewart,RB,12.5,11.709102
649,J.Hill,RB,8.0,10.634343
417,L.Miller,RB,10.0,10.632314
41,F.Gore,RB,11.5,10.370764
728,T.Rawls,RB,1.5,9.442563
276,M.Ingram,RB,4.0,9.418262


In [276]:
np.sqrt(np.abs(mean_squared_error(preds_df[preds_df["position"] == "RB"]["true"], preds_df[preds_df["position"] == 'RB']["preds"])))

5.029735434691906

## Predicting week-by-week performance
Let's see where we can get by predicting week by week performance.